In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random as rd
import csv
import numpy as np
import math

# Constants

In [ ]:
# Constants
MAX = 2e9  # maximum value of edge weight
MIN = 1  # minimum value of edge weight (not real min because we account for FACTOR)
NUM_DEC = 5  # number of decimal numbers for rounding
FACTOR = 0.9 # FACTOR of edge weight
MIN_P = 0.55 # minimum percentage
MAX_P = 0.95 # maximum percentage
SMALL = 50   # small input
MEDIUM = 100 # medium input
LARGE = 200  # large input
TEST_SIZE = 10  # use this to change the size of test cases

# maximum number of TA's / homes
SMALL_TA = 25
MEDIUM_TA = 50
LARGE_TA = 100

# Utils

In [ ]:
def write_to_file(file, string, append=False):
    if append:
        mode = 'a'
    else:
        mode = 'w'
    with open(file, mode) as f:
        f.write(string)


def write_data_to_file(file, data, separator, append=False):
    if append:
        mode = 'a'
    else:
        mode = 'w'
    with open(file, mode) as f:
        for item in data:
            f.write(f'{item}{separator}')
            
def print_edge_weights(g, n=5):
    for edge, data in g.edges.items():
        if n <= 0:
            break
        n-=1
        print(edge, data)  

# NetworkX tutorials

In [ ]:
# G = nx.Graph([(1,1), (2,2), (1,2), (1,2)])
G = nx.Graph()
G.add_node(1)
G.add_node(2)
G.add_edge(1, 2)
G.add_edge(2, 2)
A = nx.adjacency_matrix(G)
print(A.todense())

In [ ]:
G = nx.Graph()
# H = nx.path_graph(2)
# G.add_nodes_from(H)
G = nx.petersen_graph()
# pos = nx.nx_agraph.graphviz_layout(G)
# nx.draw(G, pos=pos)
# plt.show()
# plt.subplot(121)

# nx.draw(G, with_labels=True, font_weight='bold')
# plt.subplot(122)

# nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')


In [ ]:
G1 = nx.complete_graph(3)
# plt.subplot(121)
# nx.draw(G1, node_color='blue', with_labels=True)
print(list(G1.nodes()))
print(list(G1.edges()))

G2 = nx.complete_graph(range(3,6))
# plt.subplot(122)
# nx.draw(G2, node_color='red', with_labels=True)

G3 = nx.union(G1, G2)
G3.add_edge(2, 4, weight=4.7)
print(G3[4][2])
nx.draw(G3, node_color='red', with_labels=True)
print(list(G.nodes()).pop())

# Method 1: Randomly generate triangles and connect them together

In [ ]:
def add_weights(g):
    w1 = rd.uniform(MIN, MAX)
    w2 = rd.uniform(MIN_P * w1, MAX_P * w1)
    w3 = rd.uniform(MIN_P * w1, MAX_P * w1)
    ws = [w1, w2, w3]
    for _, data in g.edges.items():
        data['weight'] = round(ws.pop(), NUM_DEC)
    
def connect(G, g):
    Gnodes = list(G.nodes)
    gnodes = list(g.nodes)
    G = nx.union(G, g)
    for n in gnodes:
        idx = rd.randrange(0, len(Gnodes))
        a = Gnodes.pop(idx)
        try:
            w = nx.shortest_path_length(G, source=a, target=n) * FACTOR
        except nx.NetworkXNoPath:
            ws = [i for _, _, i in g.edges.data('weight')]
            w = rd.uniform(min(ws), max(ws)) * FACTOR
        G.add_edge(a, n, weight=round(w, NUM_DEC))
    return G
        
def gen_graph(n):
    G = nx.Graph()
    while n:
        a = n.pop()
        b = n.pop()
        c = n.pop()
        nodes = [a, b, c]
        g = nx.complete_graph(nodes)
        add_weights(g)
        if len(G.nodes) == 0:
            G = g
        else:
            G = connect(G, g)
    return G

In [ ]:
G = gen_graph(list(range(24)))
try:
    nx.shortest_path_length(G, source=2, target=3)
except nx.NetworkXNoPath:
    print("No path")
# for e in G.edges.items():
#     print(e)
nx.draw(G, node_color='red', with_labels=True)
A = nx.adjacency_matrix(G).toarray()
B = [[None] * len(A[0])] * len(A)
for i in range(len(A)):
    for j in range(len(A[0])):
        if A[i][j] == 0.0:
            B[i][j] = 'x'
        else:
            B[i][j] = A[i][j]
print(B)

# Method 2: Complete Graph with Random Weights Adjusted

Generate a complete graph with random weights associated each edge. Use floyd_warshall to produce the 
all-pairs shortest path distances. For each edge, set the length of the edge to be the length of the shortest distance between those two vertices.

Drawbacks: After adjusting the weights, some points become lying along the way between two points as their middle points. It also requires more time to generate a valid graph.

In [ ]:
# Generate 
G_final = nx.complete_graph(LARGE)
nx.draw(G_final, node_color='yellow', with_labels=True)

# print(list(G1.nodes()))
# print(list(G1.edges()))

In [ ]:
def add_weights2(g, lower, upper):
    """Update the weight of each edge to be a random weight within the bounds."""
    for edge, data in g.edges.items():
        w = rd.uniform(lower, upper)
        data['weight'] = round(w, NUM_DEC)
#         print(edge, data)        

In [ ]:
# dict1 = nx.floyd_warshall(G_final, weight='weight')
# plt.subplot(121)
# nx.draw(G_final, node_color='yellow', with_labels=True)
# print(list(G_final.nodes()))
# print(list(G_final.edges().items()))
# dict1 = nx.all_pairs_shortest_path_length(G_final)

In [ ]:
def print_shortest_pair_distance(g):
    rs = nx.floyd_warshall(g)
    for i in rs:
        for j in rs[i]:
            print ("%d\t%d\t%f" % (i, j, rs[i][j]))

In [ ]:
def fix_triangle_inequalities(g):
    done = False
    count = 0
    num_fixes = 0
    while not done:
        done = True
        rs = nx.floyd_warshall(g, weight='weight')
        print("iteration ", count)
        for edge, data in g.edges.items():
            if data['weight'] > rs[edge[0]][edge[1]]:
#                 print('change ' + str(edge) + " " + str(data['weight']) + " to " + str(rs[edge[0]][edge[1]]))
                data['weight'] = rs[edge[0]][edge[1]]
                done = False
                num_fixes += 1
#                 break # with break will lead to less fixes but takes more time to update
        count += 1
    print("total fixes: ", num_fixes)

Generate random weight for each edge.

In [ ]:
# G_final[2][3]['weight'] = 78.73409
add_weights2(G_final, MIN, MAX)
print_edge_weights(G_final)  

Fix the weight to satisfy the trangle inequalities.

In [ ]:
fix_triangle_inequalities(G_final)
print_edge_weights(G_final)  

# Method 3: Generate Random Coordinates Within Bounds

In this method, we treat the locations as a point on a 2D plane with x, y coordinates. If we connect any three locations as a triangle and set the distance between any two locations as the edge weight, then the edge weights are guaranteed to satisfy the triangle inequality because on a plane the shortest path between two points is the edge directly connect them.

Genearte random points with coordinates (x, y), where $0 < x < 10^9$ and $0 < y < 10^9$, so the maximum possible weight is $\sqrt{(10^9)^2 + (10^9)^2} = \sqrt{2 \times 10^9} < 2 \times 10^9$.

In [ ]:
def generate_random_coordinates(num_verticies, included_rects):
    """ generate a list of random coordinates which are in the included rects.
    included_rect = [[x_left, x_right, y_bot, y_top], ...]"""
    print("included_rects:", included_rects)
    coords = []
    minX = float('inf')
    minY = float('inf')
    maxX = -float('inf')
    maxY = -float('inf')
    for i in range(num_verticies):
        rect = rd.choice(included_rects)
        
        x = round(rd.uniform(rect[0], rect[1]), NUM_DEC)
        y = round(rd.uniform(rect[2], rect[3]), NUM_DEC)
        
        coords.append((x, y))
        if x < minX:
            minX = x
        if y < minY:
            minY = y
        if x > maxX:
            maxX = x
        if y > maxY:
            maxY = y
    print("minX: ", minX)
    print("minY: ", minY)
    print("maxX: ", maxX)
    print("maxY: ", maxY)
#     print(coords)
    return coords

def calculate_dist(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

In [ ]:
# Plot coordinates in scatter plot
def plot_coordinates(data):
    """data: list of tuples (x, y)"""
    data = np.array(data)
    x, y = data.T
    plt.scatter(x,y)
    plt.show()

In [ ]:
# test included_rect
coords = generate_random_coordinates(LARGE, [[0, .5*MAX, 0, .5*MAX], [.5*MAX, MAX, .5*MAX, MAX]])
plot_coordinates(coords)
# print(coords)

In [ ]:
# Generate 
G_final = nx.complete_graph(LARGE)
plt.subplot()
nx.draw(G_final, node_color='yellow', with_labels=True)

In [ ]:
def add_weights3(g, coords, num_print_msg=5):
    """Update the weight of each edge to be the distance between their location."""
    for edge, data in g.edges.items():
        data['weight'] = round(calculate_dist(coords[edge[0]], coords[edge[1]]), NUM_DEC)
        if num_print_msg > 0:
            print(str(edge) + "weight: " + str(data['weight']))
        num_print_msg -= 1

In [ ]:
add_weights3(G_final, coords)

In [ ]:
fix_triangle_inequalities(G_final)
plt.subplot()
nx.draw(G_final, node_color='yellow', with_labels=True)

# Formatting Inputs, Outputs

In [ ]:
f = open('world-cities.csv')
csv_f = csv.reader(f)
world_cities = []
for r in csv_f:
    name = r[0]
    if name.isalpha() and name != 'name':
        world_cities.append(name)
city_num = len(world_cities)
# print(world_cities)
small_list = []
medium_list = []
large_list = []
for i in range(SMALL):
    idx = rd.randrange(0, city_num)
    small_list.append(world_cities[idx])
for i in range(MEDIUM):
    idx = rd.randrange(0, city_num)
    medium_list.append(world_cities[idx])
for i in range(LARGE):
    idx = rd.randrange(0, city_num)
    large_list.append(world_cities[idx])

In [ ]:
input_file = "7\n4\nSoda"
write_to_file("input.txt", input_file)

## Save and Load a Graph Object from File Using Pickle

In [ ]:
import pickle
# save a graph obj
filename = "G_final_1.p"

plt.subplot()
nx.draw(G_final, node_color='yellow', with_labels=True)

pickle.dump(G_final, open(filename, "wb"))

In [ ]:
import pickle
G_final_copy = pickle.load( open(filename, "rb"))

plt.subplot()
nx.draw(G_final_copy, node_color='yellow', with_labels=True)